In [14]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AdversialAttackAtariEnvironment import *
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Seaquest-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/seaquest/S1/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

Discrete(18)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [15]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=False)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 200, False)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [16]:
tf.reset_default_graph()
test_method()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/seaquest/S1/checkpoints/model825673.cptk


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Run  0 Reward:  1740.0
Run  1 Reward:  1760.0
Run  2 Reward:  1740.0
Run  3 Reward:  1760.0
Run  4 Reward:  1760.0
Run  5 Reward:  1800.0
Run  6 Reward:  1720.0
Run  7 Reward:  1740.0
Run  8 Reward:  1740.0
Run  9 Reward:  1740.0
Run  10 Reward:  1760.0
Run  11 Reward:  1740.0
Run  12 Reward:  1340.0
Run  13 Reward:  1680.0
Run  14 Reward:  1740.0
Run  15 Reward:  1740.0
Run  16 Reward:  1760.0
Run  17 Reward:  1760.0
Run  18 Reward:  1740.0
Run  19 Reward:  1760.0
Run  20 Reward:  1760.0
Run  21 Reward:  1720.0
Run  22 Reward:  1720.0
Run  23 Reward:  1360.0
Run  24 Reward:  1740.0
Run  25 Reward:  1760.0
Run  26 Reward:  1740.0
Run  27 Reward:  1720.0
Run  28 Reward:  1800.0
Run  29 Reward:  1740.0
Run  30 Reward:  1760.0
Run  31 Reward:  1720.0
Run  32 Reward:  1700.0
Run  33 Reward:  1740.0
Run  34 Reward:  1760.0
Run  35 Reward:  1740.0
Run  36 Reward:  1740.0
Run  37 Reward:  1720.0
Run  38 Reward:  1760.0
Run  39 Reward:  1740.0
Run  40 Reward:  1760.0
Run  41 Reward:  1760.0
Ru